<a href="https://colab.research.google.com/github/JulianotCosta/Dictionary_IA/blob/master/Fronteira_%C3%B3tima.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
tickers = ['VALE3.SA', 'TAEE11.SA']
periodo = '1y'

Baixando Cotações e Calculando Retornos


In [29]:
import yfinance as yf
import pandas as pd
import numpy as np

import plotly.graph_objects as go

from ipywidgets import interact

In [30]:
def fix_col_names(df):
  return ['IBOV' if col == '^BVSP' else col.rstrip('.SA') for col in df.columns]

In [31]:
prices = yf.download(tickers, period=periodo, rounding=True)['Adj Close']
prices.columns = fix_col_names(prices)
prices.dropna(inplace=True)
retorno = prices.pct_change().dropna()
ativos = retorno.columns.tolist()

[*********************100%***********************]  2 of 2 completed


In [32]:
prices.tail()

,TAEE11,VALE3
Date,,
2024-09-06 00:00:00+00:00,35.82,56.68
2024-09-09 00:00:00+00:00,35.67,56.68
2024-09-10 00:00:00+00:00,35.46,56.00
2024-09-11 00:00:00+00:00,35.30,57.59
2024-09-12 00:00:00+00:00,35.00,58.11


In [33]:
retorno.tail()

,TAEE11,VALE3
Date,,
2024-09-06 00:00:00+00:00,-0.000558,-0.012544
2024-09-09 00:00:00+00:00,-0.004188,0.000000
2024-09-10 00:00:00+00:00,-0.005887,-0.011997
2024-09-11 00:00:00+00:00,-0.004512,0.028393
2024-09-12 00:00:00+00:00,-0.008499,0.009029


Criando a Carteira

In [34]:
def calcula_carteira(df, w1):
  pesos = [1-w1, w1]
  df2 = df.dot(pesos).copy()
  return df2.mean() * 252, df2.std() * np.sqrt(252)

In [38]:
from logging import captureWarnings
# Criando 100 carteiras de dois ativos com pesos variáveis
carteira = pd.DataFrame()
for i in np.linspace(0,1, 101):
  media, std = calcula_carteira(retorno,i)
  carteira.at[i,'retorno'] = media
  carteira.at[i,'volatilidade'] = std

carteira

,retorno,volatilidade
0.00,0.110906,0.129059
0.01,0.109400,0.127994
0.02,0.107893,0.126965
0.03,0.106387,0.125972
0.04,0.104881,0.125017
...,...,...
0.96,-0.033694,0.205311
0.97,-0.035200,0.207290
0.98,-0.036706,0.209277
0.99,-0.038213,0.211272


In [41]:
#carteira de volatilidade mínima
min_vol_idx = carteira['volatilidade'].idxmin()

In [42]:
# Calcula Fronteira Eficiente
if (carteira['retorno'].iloc[0] > carteira['retorno'].iloc[-1]):
  fe = carteira.loc[:min_vol_idx, :]
else:
  fe = carteira.loc[min_vol_idx:, :]

In [49]:
def gerar_grafico(w1, mostrar_curvas):
  fig = go.Figure()

  # Desenha um ponto com o retorno e a volatilidade da carteira
  fig.add_scatter(y=[carteira.iloc[[w1]]['retorno']],
                  x=[carteira.iloc[[w1]]['volatilidade']],
                  text=['CARTEIRA'],
                  mode='markers+text', name='CARTEIRA')

  # Desenha os pontos das ações individuais
  fig.add_scatter(y=carteira.iloc[[-1,0]]['retorno'],
                  x=carteira.iloc[[-1,0]]['volatilidade'],
                  text=ativos,
                  mode='markers+text', name='Ações')